In [0]:
!pip install --no-cache-dir torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html
!pip install --no-cache-dir torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html
!pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-1.4.0.html
     |████████████████████████████████| 10.6MB 89.0MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.4.0.html
     |████████████████████████████████| 15.2MB 815kB/s 
     |████████████████████████████████| 133kB 4.7MB/s 
     |████████████████████████████████| 348kB 15.2MB/s 
     |████████████████████████████████| 51kB 9.2MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.4.3-cp36-none-any.whl size=234873 sha256=4ed2f04f8805e7b12da260f8537e807b46ba8331ab8efdf9eb5eb178633b20cf
  Stored in directory: /root/.cache/pip/wheels/e2/c1/09/8693feee3f97e440d68b09abfca8b4c1e97150ace350b5003f
Successfully built torch-geometric


In [0]:
import itertools
import numpy as np
from collections import deque
import random
from scipy.stats import entropy
from datetime import datetime
import os
import networkx as nx
from copy import copy
import gym
from gym import spaces
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GatedGraphConv, global_add_pool
from torch_geometric.nn import BatchNorm
import torch.nn.functional as F
from torch.nn import Linear, Sequential, ReLU
from torch_geometric.nn import ChebConv, TopKPooling, MessagePassing, AGNNConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp, fps
from torch_geometric.data import Data, Batch
from time import sleep
import matplotlib.animation as animation
from matplotlib import rc
from IPython.display import HTML
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
class ReplayMemory:

    def __init__(self, max_size):
        '''
        The buffer is implemented as a `deque`.
        See https://docs.python.org/2/library/collections.html#collections.deque
        '''
        self.max_size = max_size
        self.buffer = deque(maxlen=max_size)

    def push(self, state, action, reward, next_state, done):
        experience = (state, action, np.array([reward]), next_state, done)
        self.buffer.append(experience)

    def sample(self, batch_size):
        '''
        Sampling method needed for experience replay.
        '''
        state_batch = []
        action_batch = []
        reward_batch = []
        next_state_batch = []
        done_batch = []

        # Randomized sampling
        batch = random.sample(self.buffer, batch_size)

        for experience in batch:
            state, action, reward, next_state, done = experience
            state_batch.append(state)
            action_batch.append(action)
            reward_batch.append(reward)
            next_state_batch.append(next_state)
            done_batch.append(done)

        return (state_batch, action_batch, reward_batch, next_state_batch, done_batch)

    def __len__(self):
        return len(self.buffer)

class GCN2(torch.nn.Module):
    def __init__(self, num_features, num_actions):
        super(GCN2, self).__init__()
        self.conv1 = GatedGraphConv(out_channels=num_features, num_layers=4, aggr='add')
        self.conv2 = GatedGraphConv(out_channels=64, num_layers=4, aggr='add')
        self.conv3 = GatedGraphConv(out_channels=128, num_layers=4, aggr='add')

        self.fc1 = Linear(in_features=128, out_features=256)
        self.fc2 = Linear(in_features=256, out_features=num_actions)

        self.bn1 = BatchNorm(num_features)
        self.bn2 = BatchNorm(64)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        x = F.elu(self.conv1(x, edge_index))
        x = self.bn1(x)
        
        x = F.elu(self.conv2(x, edge_index))
        x = self.bn2(x)
        
        x = F.elu(self.conv3(x, edge_index))

        x = global_add_pool(x, batch)
        x = F.elu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)

        return F.log_softmax(self.fc2(x), dim=-1)

class Agent:
    '''
    The agent class.
    '''
    def __init__(self, 
                 env,
                 learning_rate=5e-4,
                 gamma=0.9,
                 beta=1e-3,
                 weight_dec=5e-4,
                 buffer_size=10000):
        
        self.env = env
        self.learning_rate = learning_rate
        self.gamma = gamma
        self.beta = beta
        self.weight_dec = weight_dec
        self.replay_buffer = ReplayMemory(max_size=buffer_size)

        # Select device
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.Q_eval = GCN2(num_features=env.data.num_features, 
                          num_actions=env.action_space.n).to(self.device)
        self.Q_target = GCN2(num_features=env.data.num_features, 
                            num_actions=env.action_space.n).to(self.device)
        
        self.soft_update_target()

        # Use Adam
        self.optimizer = torch.optim.Adam(self.Q_eval.parameters(), 
                                          lr=self.learning_rate,
                                          weight_decay=self.weight_dec)
        
        # Specify loss
        self.loss_fn = torch.nn.MSELoss()
        #self.loss_fn = torch.nn.SmoothL1Loss()
        
        # Printing model & params
        print(self.Q_eval)
        trainable_params = sum(p.numel() for p in self.Q_eval.parameters() if p.requires_grad)
        print('Trainable params = {}'.format(trainable_params))
        for name, p in self.Q_eval.named_parameters():
            if p.requires_grad:
                print('\t{} : {} trainable params'.format(name, p.numel()))

    def get_action(self, state, eps=0.20):
        '''
        Epsilon-greedy strategy for action selection.
        '''
        # Select next action based on an epsilon-greedy strategy
        if np.random.rand() < eps:
            exploit_flag = False
            # Randomly sample from action space
            action = self.env.action_space.sample()
        else:
            exploit_flag = True
            # Required by torch_geometric
            state = Batch.from_data_list([state]).to(self.device)
            
            # Calculate the Q(s,a) approximation
            self.Q_eval.eval()
            with torch.no_grad():
                qvals = self.Q_eval(state)
            self.Q_eval.train()
            action = np.argmax(qvals.cpu().detach().numpy())
        return action, exploit_flag

    def compute_loss(self, batch):
        '''
        Compute loss for batch.
        '''
        # De-compose batch
        states, actions, rewards, next_states, dones = batch
        actions = torch.LongTensor(actions).to(self.device)
        rewards = torch.FloatTensor(rewards).to(self.device)
        dones = torch.FloatTensor(dones).to(self.device)
        states, next_states = Batch.from_data_list(states), Batch.from_data_list(next_states)
        states, next_states = states.to(self.device), next_states.to(self.device)
        
        self.Q_eval.train()
        self.Q_target.eval()
        
        # Calculate current Q
        curr_Q = self.Q_eval(states).gather(1, actions.unsqueeze(1))
        
        # Calculate next Q and its max
        with torch.no_grad():
            next_Q = self.Q_target(next_states)

        max_next_Q = next_Q.max(1)[0].unsqueeze(1)
        dones = dones.unsqueeze(1)
        
        # Take expectation
        expected_Q = rewards + (1-dones)*self.gamma*max_next_Q

        # Compute loss
        loss = self.loss_fn(curr_Q, expected_Q)
        return loss

    def update(self, batch_size):
        '''
        Update network parameters via SGD.
        '''
        batch = self.replay_buffer.sample(batch_size)
        loss = self.compute_loss(batch)

        self.optimizer.zero_grad()
        loss.backward()
        for param in self.Q_eval.parameters():
            param.grad.data.clamp_(-1, 1)
        self.optimizer.step()
        self.soft_update_target()
        return loss
    
    def soft_update_target(self):
        '''
        θ_target = β*θ_local + (1 - β)*θ_target
        '''
        for target_param, local_param in zip(self.Q_target.parameters(),
                                             self.Q_eval.parameters()):
            target_param.data.copy_(self.beta*local_param.data + (1-self.beta)*target_param.data)

class Environment(gym.Env):
    '''
    The graph.
    '''
    metadata = {'render.modes':[]}
    def __init__(self, path_to_map, fixed_start_node=False):
        
        # Get the adjacency matrix and the obstacles list
        state = np.load(path_to_map)
        self.adj, self.obstacles = state[0], state[1].nonzero()[0]

        # Construct networkx graph (may be useful)
        self.G = nx.from_numpy_matrix(self.adj)

        self.n_nodes = self.adj.shape[0]
        self.valid_nodes = [n for n in range(self.n_nodes) 
                            if n not in self.obstacles]
        
        #print('\t{}'.format(path_to_map))
        #print('\t\tThere are {} visitable nodes'.format(len(self.valid_nodes)))
        #print('\t\tThere are {} obstacles'.format(len(self.obstacles)))

        # Construct data (required by torch_geometric)
        self.edge_index = torch.tensor(self.adj.nonzero(), dtype=torch.long)
        self.grid_size = int(np.sqrt(self.n_nodes))
        self.pos = torch.tensor(list(itertools.product(range(self.grid_size), range(self.grid_size))))
        self.data = self.build_data()
        
        self.fixed_start_node = fixed_start_node
        # Observation space is the discrete space of all nodes
        self.observation_space = spaces.Discrete(self.n_nodes)
        
        # Action space is (0:Left, 1:Up, 2:Right, 3:Down)
        self.action_space = spaces.Discrete(4)
        
        # History of explored nodes.
        self.history = []
        return
    
    def build_data(self):
        node_features = self.init_features()
        edge_features = torch.ones(len(self.G.edges))
        return Data(x=node_features, 
                    edge_index=self.edge_index, 
                    edge_attr=edge_features, 
                    pos=self.pos)
    
    def get_n_neighs_to_visit(self, X):
        list_of_neighs = [self.get_neigh(node) for node in self.G.nodes]
        visited = [torch.where(X[n, 3] > 0)[0] for n in list_of_neighs]
        n_to_visit = [len(list_of_neighs[i]) - len(visited[i]) for i in range(self.n_nodes)]
        return torch.tensor(n_to_visit)

    def init_features(self):
        '''
        This method initializes the graph features.
        Every node has 5 features:
        (is_obstacle, n_visits, is_wall, n_neighs_to_visit, curr_occup)
        where 
        0,1 - `x,y,` are x,y coordinates of the node
        2 - `is_obstacle` is a boolean flag indicating whether the node is an obstacle
        3 - `n_visits` the number of visits for the node
        4 - `is_wall` is a boolan flag indicating wheter the node is part of the perimetral wall
        5 - `n_neighs_to_visit` is the number of neighbors yet to be visited
        6 - `curr_occup` is a boolean flag indicating whether the node is occupied by an agent
        '''
        num_feat = 7
        X = torch.zeros((self.n_nodes, num_feat), dtype=torch.float)
        # First two features are `(x,y)` coordinates
        X[:,:2] = torch.tensor([x for x in itertools.product(range(self.grid_size), range(self.grid_size))])
        # Third feature is `is_obstacle`
        X[self.obstacles, 2] = 1
        # Fifth feature is `is_wall`
        frontier = [i for i in range(self.n_nodes) if X[i,0] == 0 or X[i,0] == 9 or X[i,1] == 0 or X[i,1] == 9]
        X[frontier, 4] = 1
        # Sixth feature is `n_neighs_to_visit`
        X[:, 5] = self.get_n_neighs_to_visit(X)
        return X
    
    def standardize(self, tensor):
        means = tensor.mean(dim=1, keepdim=True)
        stds = tensor.std(dim=1, keepdim=True)
        return ((tensor - means)/stds)

    def get_visited_nodes(self):
        '''
        Returns list of visited nodes
        '''
        return torch.where(self.data.x[:, 3] > 0)[0].tolist()

    def get_history(self):
        return self.history.copy()

    def get_neigh(self, node_idx):
        return list(self.G.neighbors(node_idx))

    def map_neigh_action(self):
        '''
        Returns a dictionary (action, next state)
        that maps actions to future nodes.
        '''
        neighs = self.get_neigh(self.current_node)
        action_state = {}
        for n in neighs:
            if n + self.grid_size == self.current_node:
                action_state[1] = n
            elif n - self.grid_size == self.current_node:
                action_state[3] = n
            elif n + 1 == self.current_node:
                action_state[0] = n
            elif n - 1 == self.current_node:
                action_state[2] = n
            else:
                print('Something wrong')
                exit()
        return action_state

    def step(self, action):
        '''
        This method executes the action and returns the next state and reward. 
        It also checks if the agent reached its goal, i.e. visit every node.
        '''
        action_state = self.map_neigh_action()
        n_nodes_vis = len(self.get_visited_nodes())
        info = {}
        # If action is valid (agent does not want to exit the map)
        if action in action_state.keys():
            next_node = action_state[action]
            if next_node in self.valid_nodes:
                # Execute the transition to the next state
                self.current_node = next_node
                # Update currently occupied
                self.data.x[:, 6] = 0
                self.data.x[self.current_node, 6] = 1
                # Reward function
                if self.current_node in self.get_visited_nodes():
                    reward = -0.1*self.data.x[self.current_node, 3]
                else:
                    reward = 0.1
            else:
                reward = -0.5  - 0.1*self.data.x[self.current_node, 3]
        else:
            reward = -0.5 - 0.1*self.data.x[self.current_node, 3]
        
        # Update visit count
        self.data.x[self.current_node, 3] += 1

        # Update number of neighbors to be visited
        self.data.x[:, 5] = self.get_n_neighs_to_visit(self.data.x)

        # First reward function used, seems not to work
        #reward = (len(self.get_visited_nodes()) - n_nodes_vis)#/len(self.valid_nodes)

        # Log into history
        self.history.append(self.current_node)

        # Check if agent visited every node
        done = all(self.data.x[self.valid_nodes, 3] > 0)
        if done:
            reward += 1.
        
        return self.data.clone(), reward, done, info
            
  
    def reset(self):
        '''
        Randomly initializes the state to a valid node
        '''
        # Re-init torch_geometric `Data` object
        self.data = self.build_data()
        
        # Randomly select the first state from set of valid nodes
        self.current_node = np.random.choice(self.valid_nodes)
        if self.fixed_start_node:
            self.current_node = 0
        
        # Log into history
        self.history = []
        self.history.append(self.current_node)
        
        # Update visit count
        self.data.x[self.current_node, 3] += 1
        
        # Set currently occupied
        self.data.x[self.current_node, 6] = 1
        
        return self.data.clone()
    
    def render(self, mode='human', close=False):
        plt.clf()
        pos = [(y,9-x) for (x,y) in self.pos]
        visited = self.get_visited_nodes()
        nx.draw_networkx_edges(self.G, pos=pos)
        nx.draw_networkx_nodes(self.G, pos=pos,
                        nodelist=self.obstacles,
                        node_color='r',
                        node_size=1000)
        nx.draw_networkx_nodes(self.G, pos=pos,
                        nodelist=visited,
                        node_color='g',
                        node_size=1000)
        nx.draw_networkx_nodes(self.G, pos=pos,
                               nodelist=[self.current_node],
                               node_color='black',
                               node_size=1000)
        #nx.draw_networkx_labels(self.G, pos=pos, font_size=10, font_family='sans-serif')
        #weights = nx.get_edge_attributes(self.g,'weight')
        plt.axis('off')
        #plt.draw()
        plt.pause(0.001)
        return

In [0]:
class GraphTraversal():

    def __init__(self, g, pos, valid_nodes, obstacles, traversal):
        self.g = g
        self.pos = [(y,9-x) for (x,y) in pos]
        self.valid_nodes = valid_nodes
        self.obstacles = obstacles
        self.traversal = traversal
    
    def animate(self):
        for i, n in enumerate(self.traversal):
            #clear_output()
            fig, ax = plt.subplots(figsize=(8,4.5))
            nx.draw(self.g, pos=self.pos, node_color='b', node_size=0)
            nx.draw_networkx_nodes(self.g, self.pos, nodelist=self.obstacles, 
                                   node_color='r',
                                   node_size=600)
            nx.draw_networkx_nodes(self.g, pos=self.pos, 
                                   nodelist=self.traversal[:i], 
                                   node_color='g', 
                                   node_size=600)
            nx.draw_networkx_nodes(self.g, pos=self.pos, nodelist=[n], 
                                   node_color='black', node_size=600)
            nx.draw_networkx_labels(self.g, self.pos)
            plt.show()
            sleep(.1)

In [0]:
def purely_random(path_to_map, num_episodes=50, num_steps=25):
    '''
    This method tests a single graph in a purely random fashion.
    Returns a list, whose len is num_episodes, with percentages 
    of visited nodes.
    '''
    env = Environment(path_to_map=path_to_map)
    
    val_nodes = len(env.valid_nodes)
    vis_nodes = []

    for ep in range(num_episodes):
        state = env.reset()
        for i in range(num_steps):
            action = np.random.choice(range(4))
            next_state, reward, done, _ = env.step(action)
            state = next_state.clone()
        vis_nodes.append(env.get_history())
    
    pctg_vis_nodes = [100*len(set(v))/val_nodes for v in vis_nodes]

    return pctg_vis_nodes

@torch.no_grad()
def single_graph_test(path_to_map, path_to_model, 
                      num_episodes=50, num_steps=25):
    
    test_env = Environment(path_to_map=path_to_map)
    model = GCN2(num_features=7, num_actions=4)
    model.load_state_dict(torch.load(path_to_model))
    model.eval()

    val_nodes = len(test_env.valid_nodes)
    vis_nodes = []

    for ep in range(num_episodes):
        state = test_env.reset()
        for step in range(num_steps):
            state = Batch.from_data_list([state])
            action = torch.argmax(model(state)).item()
            next_state, reward, done, _ = test_env.step(action)
            state = next_state.clone()
        vis_nodes.append(test_env.get_history())

    pctg_vis_nodes = [100*len(set(v))/val_nodes for v in vis_nodes]
    #gt = GraphTraversal(test_env.G, test_env.pos, test_env.valid_nodes, 
    #                    test_env.obstacles, vis_nodes[best_run])
    #gt.animate()
    
    return pctg_vis_nodes

In [0]:
def test_model_directory(path_to_model, test_dir='/content/test_set', 
                         num_episodes=50, num_steps=25):
    '''
    This method tests a whole test_set with a single model.
    '''
    verb = True

    means, variances = [], []
    test_files = sorted([filename for filename in os.listdir(test_dir) if 
                  filename.endswith('.npy')])
    print('\n|      Map     | Mean |  Std | Best Run |\n|:------------:|:----:|:----:|:--------:|')
    for fnam in test_files:
        pctg_vis_nodes = single_graph_test(test_dir+'/'+fnam, path_to_model, 
                                           num_episodes=num_episodes, 
                                           num_steps=num_steps)
        #plt.clf()
        #fig = plt.figure(figsize=(16*0.7,9*0.7))
        #plt.hist(pctg_vis_nodes, color='darkred', label=path_to_model.split('/')[-1])
        #plt.title(fnam)
        #plt.legend()
        #plt.grid(linestyle='dashed')
        #plt.xlabel('Visited nodes (%)')
        #plt.show()
        
        mean, var = np.mean(pctg_vis_nodes), np.var(pctg_vis_nodes)
        if '5x5_0' not in fnam:
            means.append(mean)
            variances.append(var)
        
        if verb:
            nam = fnam.split('/')[-1].split('.')[0]
            best_run = pctg_vis_nodes.index(max(pctg_vis_nodes))
            print('| `{}` | {} | {} |   {}   |'.
                  format(nam,
                         round(mean,1), 
                         round(np.sqrt(var),1),
                         round(pctg_vis_nodes[best_run],1)))

    mean = np.mean(means)
    var = np.sum(variances) / (len(variances)**2)

    print('\n\tFINAL : MEAN = {}  STD = {}'.
          format(round(mean, 2), round(np.sqrt(var),2)))

def test_random_directory(test_dir='/content/test_set', 
                          num_episodes=50, num_steps=25):
    '''
    This method tests a whole test_set going randomly.
    '''
    verb = True

    means, variances = [], []
    test_files = sorted([filename for filename in os.listdir(test_dir) if 
                  filename.endswith('.npy')])
    print('\n|      Map     | Mean |  Std | Best Run |\n|:------------:|:----:|:----:|:--------:|')
    for fnam in test_files:
        pctg_vis_nodes = purely_random(test_dir+'/'+fnam, 
                                       num_episodes=num_episodes, 
                                       num_steps=num_steps)
        #plt.clf()
        #fig = plt.figure(figsize=(16*0.7,9*0.7))
        #plt.hist(pctg_vis_nodes, color='darkblue', label='random')
        #plt.legend()
        #plt.grid(linestyle='dashed')
        #plt.title(fnam)
        #plt.xlabel('Visited nodes (%)')
        #plt.show()
        mean, var = np.mean(pctg_vis_nodes), np.var(pctg_vis_nodes)
        if '5x5_0' not in fnam:
            means.append(mean)
            variances.append(var)

        if verb:
            nam = fnam.split('/')[-1].split('.')[0]
            best_run = pctg_vis_nodes.index(max(pctg_vis_nodes))
            print('| `{}` | {} | {} |   {}   |'.
                  format(nam,
                         round(mean,1), 
                         round(np.sqrt(var),1),
                         round(pctg_vis_nodes[best_run],1)))
    mean = np.mean(means)
    var = np.sum(variances) / (len(variances)**2)
    print('\n\tFINAL : MEAN = {}  STD = {}'.
          format(round(mean, 2), round(np.sqrt(var),2)))

In [0]:
num_episodes = 30
num_steps = 25

path_to_map = '/content/maze_5x5_2.npy'
test_dir = '/content/test_set'
path_to_models = '/content/'

print('Map = {}\nEPISODES = {} \t STEPS = {}'.format(path_to_map, num_episodes,
                                                     num_steps))

print('\nRANDOM')
test_random_directory(test_dir=test_dir, 
                      num_episodes=num_episodes, num_steps=num_steps)
print(40*'-')

models = sorted([path_to_models+filename for filename in 
                 os.listdir(path_to_models) if filename.endswith('.pt')])
for model_fname in models:
    print('\nMODEL {}'.format(model_fname))
    test_model_directory(model_fname, test_dir=test_dir, 
                         num_episodes=num_episodes, num_steps=num_steps)
    print(40*'-')


Map = /content/maze_5x5_2.npy
EPISODES = 20 	 STEPS = 25

RANDOM

|      Map     | Mean |  Std | Best Run |
|:------------:|:----:|:----:|:--------:|
| `maze_5x5_0` | 34.5 | 11.8 |   57.9   |
| `maze_5x5_1` | 32.2 | 11.2 |   55.6   |
| `maze_5x5_2` | 33.1 | 11.6 |   72.2   |
| `maze_5x5_3` | 31.1 | 10.0 |   50.0   |
| `maze_5x5_4` | 30.8 | 12.1 |   57.9   |
| `maze_5x5_5` | 32.2 | 8.7 |   55.6   |
| `maze_5x5_6` | 28.2 | 8.0 |   41.2   |
| `maze_5x5_7` | 36.9 | 12.8 |   57.1   |
| `maze_5x5_8` | 37.0 | 9.5 |   55.0   |
| `maze_5x5_9` | 36.1 | 10.8 |   54.5   |

	FINAL : MEAN = 33.08  STD = 3.55
----------------------------------------

MODEL /content/model_16_03_2020-122921.pt

|      Map     | Mean |  Std | Best Run |
|:------------:|:----:|:----:|:--------:|
| `maze_5x5_0` | 92.9 | 6.7 |   100.0   |
| `maze_5x5_1` | 66.4 | 25.4 |   83.3   |
| `maze_5x5_2` | 70.3 | 17.0 |   94.4   |
| `maze_5x5_3` | 45.8 | 24.3 |   88.9   |
| `maze_5x5_4` | 72.1 | 13.1 |   89.5   |
| `maze_5x5_5` | 55